# Harvestor

# 

In [1]:
from sickle import Sickle

In [2]:
import sickle

In [3]:
sickle = Sickle('https://www.ssoar.info/OAIHandler/request')

In [4]:
#records = sickle.ListRecords(metadataPrefix='oai_dc')
records = sickle.ListRecords(metadataPrefix='oai_genios')

In [5]:
import xmltodict
 
# It doesn't work with Python 3! Read on for the solution!
def convert(xml_file, xml_attribs=True):
    with open(xml_file) as f:
        d = xmltodict.parse(f, xml_attribs=xml_attribs)
        return d

In [6]:
counter=0
listofcounter=[]
for r in records:
    counter+=1
    listofcounter.append(r)
    if counter%10000==0:
        print(counter)

10000
20000
30000
40000


In [7]:
from lxml import etree
import json

In [8]:
llt=[]
errorls=[]
for index,item in enumerate(listofcounter):
    try:
        llt.append(eval(json.dumps(xmltodict.parse(etree.tostring(item.xml)))))
    except:
        errorls.append(index)

In [9]:
a={}
a["result"]=llt

In [10]:
import json

with open('data_harvest1.json', 'w') as fp:
    json.dump(a, fp, indent=4)

# convert harvest data to a small dict
#

In [11]:
a = json.load(open('data_harvest1.json'))

In [12]:
qualifierdict={3: 'author',15: 'issued',20: 'isbn',21: 'issn',25: 'uri',72: 'url',
 85: 'urn',104: 'pageinfo',111: 'corporateeditor',116: 'recensioneditor',118: 'recensiontitle',
 119: 'recensionseries',124: 'pid',137: 'doi'}

In [13]:
elementdict={64:"title",10:"date"}

In [14]:
counternot=0
listofrecords_h=[]
for item in range(0,len(a["result"])):
    records=a["result"][item]['record']
    if "metadata" in records.keys():
        values=records['metadata']['dublin_core']['dcvalue']
        temp={}
        list_authors_r=[]
        list_editor_r=[]
        for item_values in values:
            if '#text' in item_values.keys() and '@element' in item_values.keys():
                if 'title'==item_values['@element']:
                    if "@qualifier" in item_values.keys() and 'alternative'==item_values['@qualifier']:
                        pass
                    else:
                        temp[64]= item_values['#text']
                elif "date"==item_values['@element']:
                    temp[10]= item_values['#text']
            if '@qualifier' in item_values.keys() and '#text' in item_values.keys():
                if 'author'==item_values['@qualifier']:
                    list_authors_r.append(item_values['#text'])
                elif 'issued'==item_values['@qualifier'] :
                    temp[15]=item_values['#text']
                elif 'isbn'==item_values['@qualifier']:
                    temp[20]=item_values['#text']
                elif 'issn'==item_values['@qualifier']:
                    temp[21]=item_values['#text']
                elif 'uri'==item_values['@qualifier']:
                    temp[25]=item_values['#text']
                elif 'url'==item_values['@qualifier']:
                    temp[72]=item_values['#text']
                elif 'urn'==item_values['@qualifier']:
                    temp[85]=item_values['#text']
                elif 'pageinfo'==item_values['@qualifier']:
                    temp[104]=item_values['#text']
                elif 'corporateeditor'==item_values['@qualifier']:
                    temp[111]=item_values['#text']
                elif 'recensioneditor'==item_values['@qualifier']:
                    list_editor_r.append(item_values['#text'])
                elif 'recensiontitle'==item_values['@qualifier']:
                    temp[118]=item_values['#text']
                elif 'recensionseries'==item_values['@qualifier']:
                    temp[119]=item_values['#text']
                elif 'pid'==item_values['@qualifier']:
                    temp[124]=item_values['#text']
                elif 'doi'==item_values['@qualifier']:
                    temp[137]=item_values['#text']
                elif 'stock'==item_values['@qualifier']:
                    temp[101]=item_values['#text']
        if 25 in temp.keys():
                temp["handler"]=temp[25].split('/')[-1]
        else:
                counternot+=1
        temp[3]= list_authors_r
        temp[116]=list_editor_r
        listofrecords_h.append(temp)


In [15]:
#pd.DataFrame(listofrecords_h)

In [16]:
dictofrecords_h={}
lk=[]
for item_records_h in listofrecords_h:
    try:
        dictofrecords_h[item_records_h['handler']]=item_records_h
        lk.append(item_records_h[101])
    except:
        print(item_records_h)

{64: 'História oral entre reflexões e memórias: Revisitando o percurso de Antônio Torres Montenegro e suas trilhas metodológicas do fazer historiográfico; Entrevista', 3: ['Cavalcanti, Erinaldo Vicente', 'Silva Soares, Fagno da'], 116: [], 21: '2447-4266', 104: '426-446', 10: '2016', 111: 'Universidade Federal do Tocantins', 101: 'article', 85: 'http://dx.doi.org/10.20873/uft.2447-4266.2016v2especial1p426', 15: '2016'}


In [17]:
total_ssoar_handler=list(dictofrecords_h.keys())

In [18]:
list(set(lk))

['incollection', 'collection', 'recension', 'monograph', 'article']

In [19]:
dictofrecords_h["27498"]

{64: 'Sicherheitspolitische Herausforderungen der Regierung Obama in Asien',
 'handler': '27498',
 3: ['Godehardt, Nadine', 'Hanif, Melanie', 'Sakaeda, Ryoma'],
 116: [],
 85: 'urn:nbn:de:0168-ssoar-274987',
 104: '8',
 25: 'http://www.ssoar.info/ssoar/handle/document/27498',
 10: '2011-12-15T11:14:00Z',
 111: 'GIGA German Institute of Global and Area Studies - Leibniz-Institut für Globale und Regionale Studien, Institut für Asien-Studien',
 101: 'monograph',
 15: '2009'}

# Generate Data for process (find ssoar match out of sowiport match)

# 

In [20]:
import pandas as pd

In [21]:
match_info=pd.read_csv("source_ssoar.csv",sep=";")

In [22]:
#match_target_info=pd.read_csv("target_ssoar.csv",sep=";")

In [23]:
sowiport_info=pd.read_json("select.json")

In [24]:
sowiport_info=pd.DataFrame(sowiport_info["response"]["docs"])

In [25]:
sowiport_ssoar_info=sowiport_info[~sowiport_info["recordurl_str_mv"].isnull()]

In [26]:
def rfi(lst):
        return lst[0]

In [27]:
sowiport_ssoar_info["recordurl_str_mv"]=sowiport_ssoar_info["recordurl_str_mv"].apply(rfi)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
match_info.columns=['ref_id','newmodel_ssoar_id', 'ref_text', 'sowi_id']

In [29]:
#match_target_info.columns=['ref_id','newmodel_ssoar_id', 'ref_text', 'sowi_id']

In [30]:
sowiport_ssoar_info.columns=['sowi_id', 'recordurl_str_mv']

In [31]:
match_sowi_ssoar_info=pd.merge(match_info,sowiport_ssoar_info, on='sowi_id', how="left")

In [32]:
#match_sowi_ssoar_target_info=pd.merge(match_target_info,sowiport_ssoar_info, on='sowi_id', how="left")

In [33]:
def ret_ssoar_handler(txt):
    return txt.split("/")[-1]

In [34]:
match_sowi_ssoar_info["ssoar_match"]=match_sowi_ssoar_info[~match_sowi_ssoar_info["recordurl_str_mv"].isnull()]["recordurl_str_mv"].apply(ret_ssoar_handler)

In [35]:
#match_sowi_ssoar_target_info["ssoar_match"]=match_sowi_ssoar_target_info[~match_sowi_ssoar_target_info["recordurl_str_mv"].isnull()]["recordurl_str_mv"].apply(ret_ssoar_handler)

In [36]:
match_sowi_ssoar_info.columns=['ref_id','source_ssoar_id', 'ref_text', 'sowi_id', 'recordurl_str_mv',
       'ssoar_match']

In [37]:
#match_sowi_ssoar_target_info.columns=['ref_id','source_ssoar_id', 'ref_text', 'sowi_id', 'recordurl_str_mv','ssoar_match']

In [38]:
match_sowi_ssoar_info.head(4)

,ref_id,source_ssoar_id,ref_text,sowi_id,recordurl_str_mv,ssoar_match
0,854794,10001,Böhle (1999): Stellungnahme des Beirats zu den...,gesis-ssoar-10001,http://www.ssoar.info/ssoar/handle/document/10001,10001
1,854795,10001,Böhle (1999): Stellungnahme des Beirats zu den...,gesis-ssoar-10001,http://www.ssoar.info/ssoar/handle/document/10001,10001
2,912285,10004,"Altmann, N.; Deiß, M.; Döhl, V.; Sauer, D. (19...",csa-sa-87R1756,NaN,NaN
3,912286,10004,"Baethge, M:; Oberbeck, H. (1986): Zukunft der ...",fes-bib-134774,NaN,NaN


In [39]:
import numpy

In [40]:
embargo=list(pd.read_csv("embargoHandles19feb2018.txt")["header"].apply(numpy.int64))

In [41]:
match_sowi_ssoar_info_ne=match_sowi_ssoar_info[~match_sowi_ssoar_info['source_ssoar_id'].isin(embargo)]

In [42]:
#listproblimetic=list(set(list(match_sowi_ssoar_target_info[~match_sowi_ssoar_target_info["ssoar_match"].isnull()]["source_ssoar_id"])))

In [43]:
#listproblimetic=pd.DataFrame(listproblimetic)
#listproblimetic.to_csv("listproblimetic.csv", index=False)

In [44]:
match_info_s=match_sowi_ssoar_info_ne[['ref_id','source_ssoar_id', 'ref_text', 'sowi_id','ssoar_match']]

In [45]:
match_info_s.head(4)

,ref_id,source_ssoar_id,ref_text,sowi_id,ssoar_match
0,854794,10001,Böhle (1999): Stellungnahme des Beirats zu den...,gesis-ssoar-10001,10001
1,854795,10001,Böhle (1999): Stellungnahme des Beirats zu den...,gesis-ssoar-10001,10001
2,912285,10004,"Altmann, N.; Deiß, M.; Döhl, V.; Sauer, D. (19...",csa-sa-87R1756,NaN
3,912286,10004,"Baethge, M:; Oberbeck, H. (1986): Zukunft der ...",fes-bib-134774,NaN


# Create dictionary for ssoar ids


# 

In [40]:
metadatavalue=pd.read_csv("metadatavalue19feb2018.csv",sep=";")

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
handeler_mapper=metadatavalue[metadatavalue["metadata_field_id"]==25][["resource_id","text_value"]]

In [42]:
handeler_mapper.columns=["inter_id","handeler"]

In [43]:
handeler_mapper["handeler"]=handeler_mapper["handeler"].apply(ret_ssoar_handler)

In [44]:
handeler_mapper.head(1)

,inter_id,handeler
4128,22591,32827


In [45]:
handeler_mapper.set_index('inter_id', inplace=True)

In [46]:
hand_map_dict=handeler_mapper.to_dict()

In [47]:
hand_map_dict=hand_map_dict['handeler']

In [48]:
hand_map_dict2={}
for key, val in hand_map_dict.items():
    hand_map_dict2[val]=key

In [49]:
import numpy as np

In [50]:
def ret_map(text):
    try:
        return str(int(hand_map_dict2[str(text)]))
    except:
        return (np.nan)

In [51]:
match_info_s["source_ssoar_int_id"]=match_info_s["source_ssoar_id"].apply(ret_map)

In [52]:
match_info_s["ssoar_match_int_id"]=match_info_s["ssoar_match"].apply(ret_map)

In [53]:
match_info_s.head(1)

,ref_id,source_ssoar_id,ref_text,sowi_id,ssoar_match,source_ssoar_int_id,ssoar_match_int_id
0,854794,10001,Böhle (1999): Stellungnahme des Beirats zu den...,gesis-ssoar-10001,10001,3189,3189


In [54]:
metadatavalue.sort_values(by=["resource_id","metadata_field_id","resource_type_id"], inplace=True)
metadatavalue.fillna("nan", inplace=True)

In [55]:
def retunrn_value_for_an_id(id_str):
    tempdf=metadatavalue[metadatavalue["resource_id"]==int(id_str)][["metadata_field_id","text_value"]]
    tempdf=tempdf.groupby('metadata_field_id').apply(lambda tempdf: tempdf.to_dict(orient='list')).to_dict()
    temp_dict={}
    for key, item in tempdf.items():
        temp_dict[key]=item['text_value']
    return temp_dict

In [56]:
#internal id
example_dict=retunrn_value_for_an_id(34979)

# Create json File
#

In [46]:
ssoar_excit_sc=match_info_s

In [47]:
ssoar_excit_sc.sort_values(by="source_ssoar_id", inplace=True)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
listofonlytarget=list(set(ssoar_excit_sc[~ssoar_excit_sc["ssoar_match"].isnull()]["ssoar_match"])-set(ssoar_excit_sc[~ssoar_excit_sc["source_ssoar_id"].isnull()]["source_ssoar_id"]))

In [49]:
tsi_ls=list(set(ssoar_excit_sc[~ssoar_excit_sc["ssoar_match"].isnull()]["ssoar_match"]))
tsi_set=set(tsi_ls)
ssi_ls=list(set(ssoar_excit_sc[~ssoar_excit_sc["source_ssoar_id"].isnull()]["source_ssoar_id"]))
ssi_ls=[str(i) for i in ssi_ls]
ssi_set=set(ssi_ls)
worikinglist=list(tsi_set.intersection(ssi_set))
len(worikinglist)

614

In [50]:
listof_ssoar_id=list(set(ssoar_excit_sc[~ssoar_excit_sc["source_ssoar_id"].isnull()]["source_ssoar_id"]))

In [51]:
worikinglist=[int(i) for i in worikinglist]
listof_ssoar_id=worikinglist

In [52]:
def creat_iden_part(set_source_dict_keys,ssoar_id,source_dict):
    id_counter=0
    temp_id_list=[]
    ids_only_list=[]
    if 124 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="pmid"
        #temp_id_set["id"]=source_dict[124]
        temp_id_set["id"]=source_dict[124]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 72 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="url"
        #temp_id_set["id"]=source_dict[72][-1]
        temp_id_set["id"]=source_dict[72]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 25 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="uri"
        #temp_id_set["id"]=source_dict[25][-1]
        temp_id_set["id"]=source_dict[25]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 21 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="issn"
        #temp_id_set["id"]=source_dict[21][-1]
        temp_id_set["id"]=source_dict[21]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 20 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="isbn"
        #temp_id_set["id"]=source_dict[20][-1]
        temp_id_set["id"]=source_dict[20]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 137 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="doi"
        #temp_id_set["id"]=source_dict[137][-1]
        temp_id_set["id"]=source_dict[137]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    if 85 in list(set_source_dict_keys):
        temp_id_set={}
        temp_id_set["a"]="unique_identifier"
        temp_id_set["type"]="urn"
        #temp_id_set["id"]=source_dict[85][-1]
        temp_id_set["id"]=source_dict[85]
        ids_only_list.append("gid:01100005"+str(ssoar_id))
        temp_id_set["iri"]="gid:01100005"+str(ssoar_id)
        temp_id_set["label"]="identifier 01100005"+str(ssoar_id)+" [id/01100005"+str(ssoar_id)+"]"
        id_counter+=1
        temp_id_list.append(temp_id_set)
    return ids_only_list,temp_id_list

In [53]:
ssoar_excit_sc.columns

Index(['ref_id', 'source_ssoar_id', 'ref_text', 'sowi_id', 'ssoar_match'], dtype='object')

In [54]:
ssoar_excit_sc[ssoar_excit_sc["sowi_id"]=="not_match"].head(1)

,ref_id,source_ssoar_id,ref_text,sowi_id,ssoar_match
14042,380805,120,"Strauss, A.L. (1994). Grundlagen qualitativer ...",not_match,NaN


In [55]:
ssoar_excit_sc[ssoar_excit_sc["sowi_id"]!="error"][ssoar_excit_sc["sowi_id"]!="not_match"].head(1)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,ref_id,source_ssoar_id,ref_text,sowi_id,ssoar_match
14016,380779,120,"Klüwer, R. (1983). Agieren und Mitagieren. Psy...",dzi-solit-0023705,NaN


In [56]:
#ssoar_excit_sc["ssoar_match_int_id"]=ssoar_excit_sc["ssoar_match_int_id"].apply(str)

In [57]:
ssoar_excit_sc["ssoar_match"]=ssoar_excit_sc["ssoar_match"].apply(str)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [58]:
#wholesid=list(ssoar_excit_sc["ssoar_match"])
#wholesid=list(set(wholesid))
#wholesid=pd.DataFrame(wholesid)
#wholesid.to_csv("si.csv",index=False)

In [59]:
#dictofrecords_h

In [60]:
listof_ssoar_id_str=[str(i) for i in listof_ssoar_id]

In [61]:
kdlr=list(dictofrecords_h.keys())

In [62]:
len(list(set(listof_ssoar_id_str).intersection(set(kdlr))))

614

In [63]:
ssoar_excit_sc.drop_duplicates(["ref_text"], inplace=True)

C:\Program Files\Anaconda2\envs\python3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [64]:
len(ssoar_excit_sc)

218982

In [65]:
ssoar_excit_sc.reset_index(drop=True,inplace=True)

In [66]:
listof_ssoar_id=list(set(listof_ssoar_id))

In [67]:
import re

In [68]:
total_ssoar_handler_int=[]
count_err=0
for i in total_ssoar_handler:
    try:
        total_ssoar_handler_int.append(int(i))
    except:
        count_err+=1

In [295]:
def json_generator(listof_ssoar_id):
    list_of_br=[]
    for ssoar_id in listof_ssoar_id:
        tem_df=ssoar_excit_sc[ssoar_excit_sc["source_ssoar_id"]==ssoar_id]
        index_temp_df=list(tem_df.index)
        ref_list_be=[]
        br_temp={}
        for index_references_in_a_pdf in index_temp_df:
            be_dict={"iri":"gbe:011001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"]),
                        "a":"entry",
                        "label": "bibliographic entry 011001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+" [be/011001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+"]",
                        "content":tem_df.ix[index_references_in_a_pdf]["ref_text"]}
        
            if tem_df.ix[index_references_in_a_pdf]["ssoar_match"]!='nan' and tem_df.ix[index_references_in_a_pdf]["ssoar_match"] in total_ssoar_handler:
                    be_dict["crossref"]="gbr:0110050000"+str(tem_df.ix[index_references_in_a_pdf]["ssoar_match"])
                    
            elif tem_df.ix[index_references_in_a_pdf]["sowi_id"]!='error' and tem_df.ix[index_references_in_a_pdf]["sowi_id"]!='not_match':
                be_dict["crossref"]="gbr:0110050001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])
                br_temp1={"a":["document"], 
                          "label":"bibliographic resource 0110050001"+
                          str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+
                          " [br/0110050001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+"]"}
                br_temp1["iri"]="gbr:0110050001"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])
                list_of_br.append(br_temp1)
            else:
                be_dict["crossref"]="gbr:0110050002"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])
                br_temp1={"a":["document"], 
                          "label":"bibliographic resource 0110050002"+
                          str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+
                          " [br/0110050002"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])+"]"}
                br_temp1["iri"]="gbr:0110050002"+str(tem_df.ix[index_references_in_a_pdf]["ref_id"])
                list_of_br.append(br_temp1)
            if tem_df.ix[index_references_in_a_pdf]["ssoar_match"]!=str(ssoar_id):    
                ref_list_be.append(be_dict)
                
        if len(ref_list_be) >0:   
            br_temp["reference"]=ref_list_be
        br_temp["iri"]="gbr:0110050000"+str(ssoar_id)
        #source_dict=eval(tem_df.ix[index_references_in_a_pdf]["ssoar_source_dict"])
        #indexforssoardict=str(tem_df.ix[index_references_in_a_pdf]["source_ssoar_id"])
        source_dict=dictofrecords_h[str(ssoar_id)]

        br_temp["a"]=["document",source_dict[101].replace("incollection","inbook").replace("recension","ReviewArticle").replace("monograph","book")]

        list_source_dict_keys=list(source_dict.keys())
        setofindex_pd={111,104,116,118,119}
        set_source_dict_keys=set(list_source_dict_keys)
        checkmylenght=list(setofindex_pd.intersection(set_source_dict_keys))
        if  len(checkmylenght)>0:
            gre_dict={"iri":"gre:0110020000"+str(ssoar_id), "a": [ "generic_format", "digital_format"], "label": "resource embodiment 0110020000"+str(ssoar_id)+ " [re/0110020000"+str(ssoar_id)+ "]","mime_type": "pdf"}
            if 104 in checkmylenght:
                if '-' in source_dict[104]:
                    gre_dict["fpage"]=str(source_dict[104]).split('-')[0]
                    gre_dict["lpage"]=str(source_dict[104]).split('-')[1]
        
        
        cont_list=[]
        if 3 in list(source_dict.keys()):
            for index_aut,author_item in enumerate(source_dict[3]):
                ar_dict={"iri": "gar:0110030000"+str(index_aut)+str(ssoar_id), "a":"role",
                         "label": "agent role 0110030000"+str(index_aut)+str(ssoar_id)+" [ar/0110030000"+str(index_aut)+str(ssoar_id)+"]",       
                         "role_type": "author", 
                         "role_of": {"iri": "gra:0110040000"+str(index_aut)+str(ssoar_id), "a":"agent",
                         "label": "responsible agent 0110040000"+str(index_aut)+str(ssoar_id)+" [ra/0110040000"+str(index_aut)+str(ssoar_id)+"]",
                                }
                         }
                ais_ls=author_item.split(",")
                if len(ais_ls)>1:
                    ar_dict["role_of"]["fname"]= ais_ls[0]            
                    ar_dict["role_of"]["gname"]= ais_ls[1]
                else:
                    ar_dict["role_of"]["name"]=ais_ls[0]
                
                cont_list.append(ar_dict) 

        #br_temp["format"]= [gre_dict]
        br_temp["label"]= "bibliographic resource 0110050000"+str(ssoar_id)+" [br/0110050000"+str(ssoar_id)+"]"
        #br_temp["title"]=source_dict[64][-1]
        br_temp["title"]=source_dict[64]
        br_temp["contributor"]=cont_list
        #br_temp["year"]=source_dict[15][-1]
        year_entity = re.findall('(\d{4})',source_dict[15])
        br_temp["date"]={"value":str(year_entity[0]),"a":"year"}
        only_id_list,temp_id_list=creat_iden_part(set_source_dict_keys,ssoar_id,source_dict)
        br_temp["identifier"]=temp_id_list
        
        if "reference" in list(br_temp.keys()):
            list_of_br.append(br_temp)
    return list_of_br

In [296]:
def Generat_br_for_remained_match(cross_ref_list):
    br_list=[]
    for br_rec_new_id in cross_ref_list:
        try:
            source_dict=dictofrecords_h[br_rec_new_id]
            br_temp={"iri": "gbr:0110050000"+br_rec_new_id,
                      "a":["document",source_dict[101].replace("incollection","inbook").replace("recension","ReviewArticle").replace("monograph","book")], 
                      "label":"bibliographic resource 0110050000"+br_rec_new_id+" [br/0110050000"+br_rec_new_id+"]",
                      "title":source_dict[64]
                     }
            cont_list=[]
            if 3 in list(source_dict.keys()):
                for index_aut,author_item in enumerate(source_dict[3]):
                    ar_dict={"iri": "gar:0110030000"+str(index_aut)+br_rec_new_id, "a":"role",
                             "label": "agent role 0110030000"+str(index_aut)+br_rec_new_id+" [ar/0110030000"+str(index_aut)+br_rec_new_id+"]",       
                             "role_type": "author", 
                             "role_of": {"iri": "gra:0110040000"+str(index_aut)+br_rec_new_id, "a":"agent",
                             "label": "responsible agent 0110040000"+str(index_aut)+br_rec_new_id+" [ra/0110040000"+str(index_aut)+br_rec_new_id+"]",
                            }
                         }
                    ais_ls=author_item.split(",")
                    if len(ais_ls)>1:
                        ar_dict["role_of"]["fname"]= ais_ls[0]            
                        ar_dict["role_of"]["gname"]= ais_ls[1]
                    else:
                        ar_dict["role_of"]["name"]=ais_ls[0]
                    cont_list.append(ar_dict)
                    
            br_temp["contributor"]=cont_list
            year_entity = re.findall('(\d{4})',source_dict[15])
            br_temp["date"]={"value":str(year_entity[0]),"a":"year"}
            list_source_dict_keys=list(source_dict.keys())
            set_source_dict_keys=set(list_source_dict_keys)
            only_id_list,temp_id_list=creat_iden_part(set_source_dict_keys,int(br_rec_new_id),source_dict)
            br_temp["identifier"]=temp_id_list
            br_list.append(br_temp)
            #print(source_dict)
        except:
            pass
        #tem_br_rec["iri"]="gbr:0110050000"+str(br_rec_new_id)
        #br_temp["a"]=["document",source_dict[101].replace("incollection","inbook").replace("recension","ReviewArticle")]
    return br_list
        
        
    

In [297]:
#len(list(set(listof_ssoar_id)))

In [298]:
#len(list(set(listof_ssoar_id).intersection(set(total_ssoar_handler_int))))

In [299]:
deltaset=set(listof_ssoar_id)-set(total_ssoar_handler_int)

In [300]:
listof_ssoar_id=list(set(listof_ssoar_id)-deltaset)

In [301]:
list_of_br=json_generator(listof_ssoar_id)

In [302]:
list_iri=list(pd.DataFrame(list_of_br)["iri"])

In [303]:
df_br_list=pd.DataFrame(list_of_br)

In [304]:
cross_ref_list=[]
for item in list(df_br_list[~df_br_list["reference"].isnull()]["reference"]):
    for item1 in item:
        cross_ref_list.append(item1["crossref"])

In [305]:
cross_ref_list2=[]
for item in list(set(cross_ref_list)-set(list_iri)):  # referenced but not exist in br
    cross_ref_list2.append(item.replace("gbr:0110050000",""))

In [306]:
list_of_br_extra=Generat_br_for_remained_match(cross_ref_list2)

In [307]:
list_of_br=list_of_br_extra+list_of_br

In [308]:
from random import shuffle

In [309]:
shuffle(list_of_br)

In [310]:
br_json={}
br_json["iri"]="gbr:"
br_json["@context"]="context.json"
br_json["ReviewArticle"]="fabio:ReviewArticle"
br_json["@graph"]=[]
br_json["@graph"]=list_of_br
#br_json["excite_project_match"]={ "@id": "biro:references", "@type": "@vocab"}

In [311]:
#br_json

In [312]:
tmls=[]
for item in br_json['@graph']:
    tmls+=item["a"]
tmls=list(set(tmls))

In [313]:
tmls

['collection', 'book', 'article', 'document', 'inbook', 'ReviewArticle']

In [314]:
import json

with open('data1.json', 'w', encoding="utf8") as fp:
    json.dump(br_json, fp, indent=4,ensure_ascii=False)

In [89]:
#1-    #type of documnet (monograph:book , recension(http://purl.org/spar/fabio/ReviewArticle)
       #, incollection:inbook) ----> added to br
#4-    #crossref should be replaced ---->    
#6-    #provenance ---->
#10-    # match not in br -->done

#2-    #replace prefix ----> done
#3-    #remove dublicate ----> done
#5-    #title altrnative ----> corrected
#7-    #check records to see that content are correct? ---->  done
#8-    # remove dublication of ref text  ----> done
#9     # remove self citation (first papge of ssoar) ----> done

# br- reference - create? - target list and apply json_genrator (target-source)
#author ambiguty

#format

In [90]:
# create a table which cotain information only diff ref_id, same ssoar_id , same match_id

In [91]:
#3798162
#706291

In [92]:
#datatype():
#00 -> id
#01 -> be
#02 -> re
#03 -> ar
#04 -> ra
#05 -> br
#============================================
#corpus(0000)
#000 -> ssoar
#001 -> sowiport
#002 -> not match
#============================================
#be -> prefix(0110) datatype(01) ref_id
#####
#id -> prefix(0110) datatype(00) datatype(00) ssoar_id
#####
#ra -> prefix(0110) datatype(04) corpus(0000) index(00) ssoar_id
#####
#ar -> prefix(0110) datatype(03) corpus(0000) index(00) ssoar_id
#####
#re -> prefix(0110) datatype(02) corpus(0000) ssoar_id
#####
#br -> prefix(0110) datatype(05) corpus(0000) (ssoar_id/ref_id)
#####

# Check data 
#

In [93]:
dataindf=pd.DataFrame(br_json['@graph'])

In [94]:
iri_df_br_dub=dataindf.duplicated("iri")

In [95]:
iri_df_br_dub=pd.DataFrame(iri_df_br_dub)

In [96]:
iri_df_br_dub[iri_df_br_dub[0]==True] #good :)   means not reapeted id and item

,0


In [97]:
len(dataindf)==len(dataindf[dataindf["iri"].str.contains("gbr:")]) # good :) all items are br

True

In [98]:
len(worikinglist)

614

In [99]:
len(dataindf[~dataindf["identifier"].isnull()])

873

In [100]:
len(worikinglist)==len(dataindf[~dataindf["identifier"].isnull()]) # good :) number of item with identifierl are equal to working list 

False

In [101]:
len(dataindf[~dataindf["year"].isnull()])

873

In [102]:
dataindf.columns

Index(['a', 'contributor', 'identifier', 'iri', 'label', 'reference', 'title',
       'year'],
      dtype='object')

In [103]:
len(dataindf[~dataindf["title"].isnull()]) # good

873

In [104]:
len(dataindf[~dataindf["reference"].isnull()]) # good

561

In [105]:
len(dataindf[~dataindf["label"].isnull()]) # good

33927

In [106]:
len(dataindf[~dataindf["iri"].isnull()]) # good

33927

In [107]:
len(dataindf[~dataindf["identifier"].isnull()]) # good 

873

In [108]:
len(dataindf[~dataindf["contributor"].isnull()]) # good

873

In [109]:
len(dataindf[~dataindf["a"].isnull()]) # good

33927

In [110]:
listofsum=[]
for item in list(dataindf["a"]):
    listofsum+=item
set(listofsum) # good

{'ReviewArticle', 'article', 'collection', 'document', 'inbook', 'monograph'}

In [111]:
listofsum=[]
for item in list(dataindf[~dataindf["contributor"].isnull()]["contributor"]):
    listofsum+=item


In [112]:
conterbiuter_list=pd.DataFrame(listofsum)

In [113]:
iri_aut_list=pd.DataFrame(conterbiuter_list.duplicated("iri"))

In [114]:
iri_aut_list[iri_aut_list[0]==True] # good not reapition in author ids

,0


In [115]:

ra_list_iri=pd.DataFrame(list(conterbiuter_list["role_of"]))["iri"]

In [116]:
ra_list_iri=pd.DataFrame(ra_list_iri)

In [117]:
ra_list_iri_dublicate=ra_list_iri.duplicated("iri")

In [118]:
ra_list_iri_dublicate=pd.DataFrame(ra_list_iri_dublicate)

In [119]:
ra_list_iri_dublicate[ra_list_iri_dublicate[0]==True] # good no dublicate in ra ids

,0


In [120]:
#dataindf
ra_df=pd.DataFrame(list(conterbiuter_list["role_of"]))

In [121]:
ra_df[~ra_df["name"].isnull()]

,a,fname,gname,iri,label,name
27,agent,NaN,NaN,gra:0110040000010020,responsible agent 0110040000010020 [ra/0110040...,ISF München
930,agent,NaN,NaN,gra:0110040000238407,responsible agent 0110040000238407 [ra/0110040...,Markus Schatten
982,agent,NaN,NaN,gra:0110040000012545,responsible agent 0110040000012545 [ra/0110040...,Franke. Sabine


In [122]:
set(list(dataindf[~dataindf["year"].isnull()]["year"]))

{'1887',
 '1900',
 '1904',
 '1929',
 '1934',
 '1974',
 '1975',
 '1976',
 '1978',
 '1979',
 '1980',
 '1981',
 '1983',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017'}

In [123]:
dataindf[dataindf["iri"]=="gbr:011005000051629"]

,a,contributor,identifier,iri,label,reference,title,year
12493,"[document, monograph]",[{'label': 'agent role 0110030000051629 [ar/01...,[{'id': 'http://www.ssoar.info/ssoar/handle/do...,gbr:011005000051629,bibliographic resource 011005000051629 [br/011...,[{'label': 'bibliographic entry 011001492303 [...,Kinderlosigkeit und Kinderreichtum: Zwei Phäno...,2015


In [124]:
len(dataindf[~dataindf["year"].isnull()]["year"])

873

In [125]:
len(dataindf[dataindf["iri"].str.contains("gbr:0110050000")])

873

In [129]:
lenref=pd.DataFrame(dataindf[~dataindf["reference"].isnull()]["reference"].apply(len))

In [130]:
len(lenref[lenref["reference"]==1])

5

In [162]:
dataindf.ix[33416]["identifier"]

[{'a': 'unique_identifier',
  'id': 'http://www.ssoar.info/ssoar/handle/document/51413',
  'iri': 'gid:0110000551413',
  'label': 'identifier 0110000551413 [id/0110000551413]',
  'type': 'uri'},
 {'a': 'unique_identifier',
  'id': '2079-8555',
  'iri': 'gid:0110000551413',
  'label': 'identifier 0110000551413 [id/0110000551413]',
  'type': 'issn'},
 {'a': 'unique_identifier',
  'id': 'http://dx.doi.org/10.5922/2079-8555-2016-1-6',
  'iri': 'gid:0110000551413',
  'label': 'identifier 0110000551413 [id/0110000551413]',
  'type': 'doi'},
 {'a': 'unique_identifier',
  'id': 'urn:nbn:de:0168-ssoar-51413-4',
  'iri': 'gid:0110000551413',
  'label': 'identifier 0110000551413 [id/0110000551413]',
  'type': 'urn'}]

In [163]:
dataindf.ix[33416]

a                                            [document, article]
contributor    [{'label': 'agent role 0110030000051413 [ar/01...
identifier     [{'id': 'http://www.ssoar.info/ssoar/handle/do...
iri                                          gbr:011005000051413
label          bibliographic resource 011005000051413 [br/011...
reference      [{'label': 'bibliographic entry 011001221417 [...
title          Local border traffic as an efficient tool for ...
year                                                        2016
Name: 33416, dtype: object

In [131]:
def repstr2(tz):
    return tz.replace("gbr:0110050000","")

In [132]:
def firstposinlist(llssstt):
    return(llssstt[0]["crossref"])

In [133]:
lenref["reference"].mean()

60.01604278074866

In [179]:
comparisiontwodfssoar_refs=pd.DataFrame(dataindf.ix[list(lenref[lenref["reference"]==1].index)]["iri"])["iri"]==pd.DataFrame(dataindf.ix[list(lenref[lenref["reference"]==1].index)]["reference"].apply(firstposinlist))["reference"]

In [180]:
comparisiontwodfssoar_refs #good

2052     False
12051    False
12504    False
26245    False
33416    False
dtype: bool

In [181]:
listoflenonereferences=['34837','32806','51240','51276','24674','51300','51312','51313','32883','51327','37015','32930','51369','51370','51371','51375','51377','51404','51407','35048','51461','35080','10604','51573','51574','6533','10768','23152','43643','33411','19091','21185','21187','45773','41729','41756','25586','50326','48320','28268','28269','28276','36524','10001','51045','10091','32659','32673','32677','10182','34808']

In [137]:
#dataindf.ix[6412]["iri"]

In [138]:
#list(dataindf[~dataindf["identifier"].isnull()][["iri","title","reference"]].index)[20]

In [139]:
#dataindf[~dataindf["identifier"].isnull()][["iri","title","reference"]].ix[763]

In [140]:
#dataindf[~dataindf["identifier"].isnull()][["iri","title","reference"]].ix[763]["iri"]

In [141]:
#dataindf[~dataindf["identifier"].isnull()][["iri","title","reference"]].ix[763]["reference"]

In [142]:
#print(dataindf[~dataindf["year"].isnull()][["iri","title","reference"]].sort_values("iri")["title"].ix[29673])

In [143]:
#dataindf[~dataindf["reference"].isnull()]

In [144]:
#dataindf[~dataindf["year"].isnull()][["iri","title","reference"]].sort_values("iri")

In [182]:
list_iri=list(set(dataindf["iri"]))

In [183]:
cross_ref_list=[]
for item in list(dataindf[~dataindf["reference"].isnull()]["reference"]):
    for item1 in item:
        cross_ref_list.append(item1["crossref"])

In [184]:
cross_ref_list=list(set(cross_ref_list))

In [185]:
len(list(set(cross_ref_list).intersection(set(list_iri))))

33527

In [186]:
len(list(set(cross_ref_list)))

33527

In [187]:
len(list(set(cross_ref_list)-set(list_iri)))  # referenced but not exist in br

0

In [188]:
len(set(list_iri)-set(cross_ref_list))      # in br but not referenced -- it is ok since we have container which is not part of matches

400

In [189]:
list_be=[]
for item in list(dataindf[~dataindf["reference"].isnull()]["reference"]):
    for item1 in item:
        list_be.append(item1["iri"])

In [190]:
len(list_be)

33669

In [326]:
#dataindf[~dataindf["reference"].isnull()]

In [208]:
#dataindf[~dataindf["reference"].isnull()].ix[33887]["identifier"]

In [207]:
#dataindf[~dataindf["reference"].isnull()][["iri","title","year"]]

In [206]:
#dataindf[~dataindf["reference"].isnull()].ix[33887]["reference"]

In [440]:
#33669 (be) reference strings from 561 papers   ---- 873 br with metadata in 33927 br